## Doc2Vec을 통한 유사한 토픽의 기사 분류 및 TfidfVectorizer를 통한 논조 분류  
- Doc2Vec을 이용하여 각 뉴스기사의 토큰들을 벡터화하여, 특정 키워드와 관련된 뉴스 토픽을 분류.
- 하나의 토픽으로 분류된 기사를 TfidfVectorizer를 이용하여 세부적인 기사의 논조 분류

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re
import json

from konlpy.tag import Mecab
import nltk

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec, Phrases

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

## 1. 전처리

### 기사별 불용어 처리를 위한 불용어 사전

In [2]:
stop_words = '아,휴,아이구,아이쿠,아이고,어,나,우리,저희,따라,의해,을,를,에,의,가,으로,로,에게,뿐이다,의거하여,근거하여,입각하여,기준으로,\
예하면,예를 들면,예를 들자면,저,소인,소생,저희,지말고,하지마,하지마라,다른,물론,또한,그리고,비길수 없다,해서는 안된다,뿐만 아니라,만이 아니다,\
만은 아니다,막론하고,관계없이,그치지 않다,그러나,그런데,하지만,든간에,논하지 않다,따지지 않다,설사,비록,더라도,아니면,만 못하다,하는 편이 낫다,\
불문하고,향하여,향해서,향하다,쪽으로,틈타,이용하여,타다,오르다,제외하고,이 외에,이 밖에,하여야,비로소,한다면 몰라도,외에도,이곳,여기,부터,기점으로,\
따라서,할 생각이다,하려고하다,이리하여,그리하여,그렇게 함으로써,하지만,일때,할때,앞에서,중에서,보는데서,으로써,로써,까지,해야한다,일것이다,반드시,\
할줄알다,할수있다,할수있어,임에 틀림없다,한다면,등,등등,제,겨우,단지,다만,할뿐,딩동,댕그,대해서,대하여,대하면,훨씬,얼마나,얼마만큼,얼마큼,남짓,여,\
얼마간,약간,다소,좀,조금,다수,몇,얼마,지만,하물며,또한,그러나,그렇지만,하지만,이외에도,대해 말하자면,뿐이다,다음에,반대로,반대로 말하자면,이와 반대로,\
바꾸어서 말하면,바꾸어서 한다면,만약,그렇지않으면,까악,툭,딱,삐걱거리다,보드득,비걱거리다,꽈당,응당,해야한다,에 가서,각,각각,여러분,각종,각자,제각기,\
하도록하다,와,과,그러므로,그래서,고로,한,까닭에,하기,때문에,거니와,이지만,대하여,관하여,관한,과연,실로,아니나다를가,생각한대로,진짜로,한적이있다,\
하곤하였다,하,하하,허허,아하,거바,와,오,왜,어째서,무엇때문에,어찌,하겠는가,무슨,어디,어느곳,더군다나,하물며,더욱이는,어느때,언제,야,이봐,어이,여보시오\
,흐흐,흥,휴,헉헉,헐떡헐떡,영차,여차,어기여차,끙끙,아야,앗,아야,콸콸,졸졸,좍좍,뚝뚝,주룩주룩,솨,우르르,그래도,또,그리고,바꾸어말하면,바꾸어말하자면,\
혹은,혹시,답다,및,그에 따르는,때가 되어,즉,지든지,설령,가령,하더라도,할지라도,일지라도,지든지,몇,거의,하마터면,인젠,이젠,된바에야,된이상,만큼\t어찌됏든\
,그위에,게다가,점에서 보아,비추어 보아,고려하면,하게될것이다,일것이다,비교적,좀,보다더,비하면,시키다,하게하다,할만하다,의해서,연이서,이어서,잇따라,\
뒤따라,뒤이어,결국,의지하여,기대여,통하여,자마자,더욱더,불구하고,얼마든지,마음대로,주저하지 않고,곧,즉시,바로,당장,하자마자,밖에 안된다,하면된다,그래,\
그렇지,요컨대,다시 말하자면,바꿔 말하면,즉,구체적으로,말하자면,시작하여,시초에,이상,허,헉,허걱,바와같이,해도좋다,해도된다,게다가,더구나,하물며,와르르,\
팍,퍽,펄렁,동안,이래,하고있었다,이었다,에서,로부터,까지,예하면,했어요,해요,함께,같이,더불어,마저,마저도,양자,모두,습니다,가까스로,하려고하다,즈음하여,\
다른,다른 방면으로,해봐요,습니까,했어요,말할것도 없고,무릎쓰고,개의치않고,하는것만 못하다,하는것이 낫다,매,매번,들,모,어느것,어느,로써,갖고말하자면,\
어디,어느쪽,어느것,어느해,어느,년도,라 해도,언젠가,어떤것,어느것,저기,저쪽,저것,그때,그럼,그러면,요만한걸,그래,그때,저것만큼,그저,이르기까지,\
할 줄 안다,할 힘이 있다,너,너희,당신,어찌,설마,차라리,할지언정,할지라도,할망정,할지언정,구토하다,게우다,토하다,메쓰겁다,옆사람,퉤,쳇,의거하여,\
근거하여,의해,따라,힘입어,그,다음,버금,두번째로,기타,첫번째로,나머지는,그중에서,견지에서,형식으로 쓰여,입장에서,위해서,단지,의해되다,하도록시키다,\
뿐만아니라,반대로,전후,전자,앞의것,잠시,잠깐,하면서,그렇지만,다음에,그러한즉,그런즉,남들,아무거나,어찌하든지,같다,비슷하다,예컨대,이럴정도로,어떻게,\
만약,만일,위에서 서술한바와같이,인 듯하다,하지 않는다면,만약에,무엇,무슨,어느,어떤,아래윗,조차,한데,그럼에도 불구하고,여전히,심지어,까지도,조차도,\
하지 않도록,않기 위하여,때,시각,무렵,시간,동안,어때,어떠한,하여금,네,예,우선,누구,누가 알겠는가,아무도,줄은모른다,줄은 몰랏다,하는 김에,겸사겸사,\
하는바,그런 까닭에,한 이유는,그러니,그러니까,때문에,그,너희,그들,너희들,타인,것,것들,너,위하여,공동으로,동시에,하기 위하여,어찌하여,무엇때문에,붕붕,\
윙윙,나,우리,엉엉,휘익,윙윙,오호,아하,어쨋든,만,못하다,하기보다는,차라리,하는 편이 낫다,흐흐,놀라다,상대적으로 말하자면,마치,아니라면,쉿,\
그렇지 않으면,그렇지 않다면,안 그러면,아니었다면,하든지,아니면,이라면,좋아,알았어,하는것도,그만이다,어쩔수 없다,하나,일,일반적으로,일단,한켠으로는,\
오자마자,이렇게되면,이와같다면,전부,한마디,한항목,근거로,하기에,아울러,하지 않도록,않기 위해서,이르기까지,이 되다,로 인하여,까닭으로,이유만으로,\
이로 인하여,그래서,이 때문에,그러므로,그런 까닭에,알,수,있다,결론을,낼,수,있다,으로,인하여,있다,어떤것,관계가 있다,관련이 있다,연관되다,어떤것들,\
에 대해,이리하여,그리하여,여부,하기보다는,하느니,하면 할수록,운운,이러이러하다,하구나,하도다,다시말하면,다음으로,에 있다,에 달려 있다,우리,우리들,\
오히려,하기는한데,어떻게,어떻해,어찌됏어,어때,어째서,본대로,자,이,이쪽,여기,이것,이번,이렇게말하자면,이런,이러한,이와 같은,요만큼,요만한 것,\
얼마,안,되는,것,이만큼,이 정도의,이렇게,많은,것,이와 같다,이때,이렇구나,것과 같이,끼익,삐걱,따위,와 같은 사람들,부류의 사람들,왜냐하면,중의하나,\
오직,오로지,에 한하다,하기만 하면,도착하다,까지 미치다,도달하다,정도에 이르다,할 지경이다,결과에 이르다,관해서는,여러분,하고 있다,한 후,혼자,자기,\
자기집,자신,우에,종합한것과같이,총적으로 보면,총적으로 말하면,총적으로,대로 하다,으로서,참,그만이다,할 따름이다,쿵,탕탕,쾅쾅,둥둥,봐,봐라,아이야,아니,\
와아,응,아이,참나,년,월,일,령,영,일,이,삼,사,오,육,륙,칠,팔,구,이천육,이천칠,이천팔,이천구,하나,둘,셋,넷,다섯,여섯,일곱,여덟,아홉,령,영,\
하루,이틀,사흘,나흘,닷새,엿새,이흐레,여드레,아흐레,열흘,변호사,회계사,사장,본부장,검사,판사,영역,버튼,기자,로또복권,동행복권'

In [3]:
stop_words = stop_words.split(',')

### 뉴스 기사 전처리를 위한 함수

In [4]:
# 문서의 토큰화를 위해서 한국어만을 남겨둠.
def get_only_korean_words(text):
    return re.sub('[^가-힣 ]',' ', text)

# 문서를 명사로 토큰화
def noun_tokenizer(text):
    nouns = Mecab().nouns(text)
    nouns = [noun for noun in nouns if noun not in stop_words]
    return [noun for noun in nouns if (len(noun) > 1)]

### 뉴스 기사 불러오기 (내일신문, 서울신문, 한겨례신문, 중앙일보, 경향신문)

In [5]:
df1 = pd.read_csv('./tomorrow2.csv', index_col=0)
df2 = pd.read_csv('./seoul.csv', index_col=0)
df3 = pd.read_csv('./han_20211101.csv', index_col=0)
df4 = pd.read_csv('./joong_20211101.csv', index_col=0)
df5 = pd.read_csv('./df_kyunghyang.csv', index_col=0)

main_df = pd.concat([df1, df2, df3, df4, df5])

In [6]:
del df1, df2, df3, df4, df5

In [7]:
main_df.isnull().sum()

일자       0
제목       0
URL      0
본문     663
dtype: int64

### 토픽 분류에 불필요한 기사 삭제  
- 글자수가 200개 미만인 기사 삭제
- 사진 중심의 기사(제목에 포토가 포함된) 삭제

In [8]:
main_df.dropna(inplace = True)
main_df['deleted'] = main_df['본문'].apply(lambda x : len(x) < 200).map({True : np.nan, False : 0})
main_df['delete_photo'] = main_df['제목'].apply(lambda x : '포토' in x).map({True : np.nan, False : 0})
main_df.dropna(inplace = True)

In [9]:
main_df = main_df.reset_index(drop=True)

### 토큰화를 위한 전처리(한국어만 가져오기)

In [10]:
main_df['본문'] = main_df['본문'].apply(get_only_korean_words)

### 각 기사를 명사 단어로 토큰화

In [11]:
%%time
main_df['본문키워드'] = main_df['본문'].apply(noun_tokenizer)

CPU times: user 4min 54s, sys: 1min 7s, total: 6min 1s
Wall time: 6min 1s


In [12]:
main_df

,일자,제목,URL,본문,deleted,delete_photo,본문키워드
0,20211101,유동규 651억원 배임 혐의로 추가 기소,http://www.naeil.com/news_view/?id_art=403349,검찰이 유동규 전 성남도시개발공사 기획본부장을 특정경제범죄가중처벌법상 억원대 ...,0.0,0.0,"[검찰, 유동규, 성남, 도시, 개발, 공사, 기획, 본부, 특정, 경제, 범죄, ..."
1,20211101,"대장동 개발 특혜 로비 의혹 사건, 권력형 범죄로 수사 중심 이동하나",http://www.naeil.com/news_view/?id_art=403344,황무성 전 성남도시개발공사 사장의 사퇴 강요 의혹이 담긴 녹취록이 공개되면서 대...,0.0,0.0,"[황무성, 성남, 도시, 개발, 공사, 사퇴, 강요, 의혹, 취록, 공개, 대장동,..."
2,20211101,김만배 남 욱 영장청구 임박,http://www.naeil.com/news_view/?id_art=403346,대장동 개발 의혹의 핵심 인물인 김만배 화천대유 대주주와 천화동인 호 실소유주인 ...,0.0,0.0,"[대장동, 개발, 의혹, 핵심, 인물, 김만배, 화천, 대유, 대주주, 천화, 동인..."
3,20211101,"[기후 위기 속에서 태어난 어린이들] ""2살 아동, 조부모보다 6.8배 폭염 겪어""",http://www.naeil.com/news_view/?id_art=403347,기후위기가 아동 권리에 심각한 영향을 미치고 있다는 연구결과가 나왔다 기후변화에 ...,0.0,0.0,"[기후, 위기, 아동, 권리, 영향, 연구, 결과, 기후, 변화, 파격, 대책, 실..."
4,20211101,"대법, 초등생 관자놀이 누른 교사 무죄 확정",http://www.naeil.com/news_view/?id_art=403341,초등학생이 약속을 지키지 않았다며 주먹으로 관자놀이를 누르거나 소란을 피우는 모습...,0.0,0.0,"[초등학생, 약속, 주먹, 관자놀이, 소란, 모습, 부모, 휴대, 전화, 동영상, ..."
...,...,...,...,...,...,...,...
92376,20210430,"삼성생명 몰아줘 경영권 공고히 전자, 세금 부담 고려 법정 배분",http://news.khan.co.kr/kh_news/khan_art_view.h...,이건희 회장 유가족 상속 지분 배분 내역 공개 이재용 부...,0.0,0.0,"[이건희, 회장, 유가족, 상속, 지분, 배분, 내역, 공개, 이재용, 부회장, 그..."
92377,20210430,"이재용 부회장, 삼성전자 지배력 강화됐다",http://news.khan.co.kr/kh_news/khan_art_view.h...,이건희 삼성생명 지분 절반 상속삼성 전자 물산은 법정비율대로 ...,0.0,0.0,"[이건희, 삼성생명, 지분, 절반, 속삼, 물산, 법정, 비율, 이재용, 삼성전자,..."
92378,20210430,"[사설] 131번째 노동절, 코로나 해고자와 노동 홀대의 아픔 새겨야",http://news.khan.co.kr/kh_news/khan_art_view.h...,노동절을 하루 앞둔 일 서울고용복지센터 앞에서 아시아나케이오 노동...,0.0,0.0,"[노동절, 서울, 고용, 복지, 센터, 아시아, 케이오, 노동자, 단식, 농성, 김..."
92379,20210430,공수처의 조용한 ‘백일맞이’ 한 방보다 신뢰가 우선,http://news.khan.co.kr/kh_news/khan_art_view.h...,수사 공정성 논란에 외부 메시지 생략 이달 호 수사 착수 가능성법리 적용 까...,0.0,0.0,"[수사, 공정, 논란, 외부, 메시지, 생략, 이달, 수사, 착수, 가능, 법리, ..."


### Gensim의 Phrases 모듈을 이용하여 유사 토큰을 묶어줌
- 명사를 중심으로 단순히 Tokenizing 된 토큰들 중 자주 쓰이는 단어로 토큰을 묶어줌.(bigram이라 표현)
- ex. [보궐, 선거]와 같이 하나의 단어로 쓰이는 토큰을 묶어줌 (보궐, 선거 -> 보궐선거)

In [13]:
%%time
bigram = Phrases(sentences=main_df['본문키워드'], min_count=30, threshold=100, delimiter='')

CPU times: user 16 s, sys: 172 ms, total: 16.1 s
Wall time: 16.1 s


In [14]:
bigram.vocab

{'검찰': 44224,
 '유동규': 2026,
 '검찰유동규': 95,
 '성남': 7592,
 '유동규성남': 976,
 '도시': 15585,
 '성남도시': 2331,
 '개발': 42662,
 '도시개발': 3380,
 '공사': 12697,
 '개발공사': 2598,
 '기획': 11073,
 '공사기획': 943,
 '본부': 19178,
 '기획본부': 1132,
 '특정': 7932,
 '본부특정': 42,
 '경제': 37783,
 '특정경제': 653,
 '범죄': 14914,
 '경제범죄': 1038,
 '중처': 692,
 '범죄중처': 664,
 '법상': 1654,
 '중처법상': 383,
 '배임': 2165,
 '법상배임': 139,
 '혐의': 33596,
 '배임혐의': 877,
 '적용': 22005,
 '혐의적용': 1179,
 '추가': 20927,
 '적용추가': 43,
 '기소': 11204,
 '추가기소': 199,
 '기소검찰': 317,
 '화천': 7904,
 '검찰화천': 133,
 '대유': 7836,
 '화천대유': 7758,
 '대주주': 2160,
 '대유대주주': 859,
 '김만배': 1989,
 '대주주김만배': 839,
 '천화': 1159,
 '김만배천화': 35,
 '동인': 1139,
 '천화동인': 1066,
 '실소유주': 446,
 '동인실소유주': 179,
 '실소유주성남': 1,
 '전략': 15317,
 '공사전략': 53,
 '사업': 47612,
 '전략사업': 172,
 '실장': 7363,
 '사업실장': 47,
 '정민용': 186,
 '실장정민용': 2,
 '구속': 13101,
 '정민용구속': 3,
 '영장': 6905,
 '구속영장': 3692,
 '청구': 7995,
 '영장청구': 1170,
 '서울': 82092,
 '청구서울': 47,
 '중앙': 21229,
 '서울중앙': 4458,
 '지방': 14949,
 '중앙지방': 785,
 '검찰청': 94

### 문서 분석을 위해서 토큰들을 bigram화 하여 리스트에 담아줌.
- 각 기사들의 token들을 bigram의 key로 주면 ( ex. bigram[['보궐', '선거']] )  토큰을 bigram으로 만들어줌

In [15]:
%%time
# bigram화 된 토큰들이 담길 리스트
bi_token_list = []

# 기사의 각 토큰(키워드)들을 bigram화 하여 리스트에 추가. (이 과정에서 stop_words를 다시 제거함.)
for tokens in main_df['본문키워드']:
    bi_token_list.append([token for token in bigram[tokens] if token not in stop_words])

CPU times: user 2min 34s, sys: 240 ms, total: 2min 34s
Wall time: 2min 34s


### Doc2Vec 적용을 위해 분석에 적합한 형식으로 바꿔줌.
- Doc2Vec의 경우 분석을 위한 토큰들이 단순히 리스트가 아닌 TaggedDocument 형식에 담겨야 함.
- TaggedDocument는 (토큰리스트, 태그)와 같은 형식으로 구성되어 있으며, 태그에는 해당 문서를 대표하는 단어를 넣어줌

In [18]:
# bigram화된 토큰들을 서로 다른 태그를 주어서 리스트에 담음.
# ex. TaggedDocument(['보궐', '선거', '개표', ... ], 태그(여기서는 단순히 숫자를 넣음))
documents = [TaggedDocument(bi_token_list[i], [i]) for i in range(len(bi_token_list))]

## 2. Doc2Vec 모델 생성
- vector_size는 뉴스를 몇 개의 feature를 사용하여 vector로 만들 것인지. (여기서는 100개의 feature로 구분하도록 지정)
- window는 뉴스 내의 토큰을 분석할 때 특정 토큰 주위 몇 개의 토큰를 함께 분석할 것인지. (여기서는 토큰 앞 뒤 각각 5개의 토큰을 함께 분석)
- min_count는 최소 몇 개의 뉴스에 나온토큰어들을 분석에 포함시킬 것인지. (10개 미만의 뉴스에만 있는 토큰 분석에서 제외)
- epochs는 모델 생성시 학습하는 횟수.

In [19]:
%%time
model = Doc2Vec(documents, vector_size=100, window=5, workers=4, min_count=10, epochs=40)

CPU times: user 16min 29s, sys: 14.7 s, total: 16min 44s
Wall time: 4min 44s


In [20]:
# 모델의 토큰 빌드
model.build_vocab(documents)

In [21]:
%%time
# 모델 학습 시작
model.train(documents, epochs=40, total_examples=model.corpus_count)

CPU times: user 16min 20s, sys: 16.7 s, total: 16min 37s
Wall time: 4min 45s


In [22]:
# model.save('./doc2vec_category.model')

In [23]:
# 전체 기사수 * 벡터사이즈수 만큼의 데이터가 만들어짐.
model.dv.vectors.shape

(92381, 100)

In [164]:
pd.DataFrame(model.dv.vectors)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.104162,-0.191747,-2.939924,2.722653,-2.592557,0.054685,0.437899,0.849698,-1.125244,-0.108780,...,-3.287439,-1.397832,-1.272927,-0.589260,-1.566129,-1.334783,-1.786013,0.567148,0.581725,0.218318
1,-1.641108,-1.389688,-0.936844,0.280835,-1.668678,1.199311,-0.201601,-1.928480,-1.052376,-1.243813,...,-0.489497,-2.726747,0.412430,0.660911,0.402406,1.234063,0.473058,-0.435122,0.260539,3.071498
2,1.611276,-2.065083,-2.110292,2.042059,-2.997705,-0.911876,0.564203,-1.117054,-1.506021,-0.400390,...,0.247009,-1.272441,0.075255,0.889493,0.377164,1.501910,0.065173,-0.705197,1.424334,1.539668
3,1.747245,-0.362899,0.678538,2.258787,-1.361518,-1.341774,-3.073230,3.482631,0.289915,1.259513,...,1.137347,1.041922,-2.034146,0.590830,1.986360,0.305679,-0.619829,-0.494725,-2.679683,1.825948
4,1.685745,0.995343,-2.224836,-0.555703,-1.579352,2.616947,0.172720,-1.915032,0.652628,1.561297,...,2.150585,2.060230,1.496731,0.680056,3.853770,-0.411563,2.286780,1.409301,2.581968,-1.846351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92376,-1.827666,-0.687670,-1.599437,-0.688014,-4.995001,1.305958,-0.690995,-0.505477,-0.100108,0.363208,...,3.242633,2.150751,-2.150522,-2.572529,3.591613,-2.962949,-3.840430,-1.721985,1.962202,1.692808
92377,-0.636449,1.670100,-1.897273,-0.812208,-2.991879,1.836629,-1.052717,0.530309,-0.847700,1.926625,...,0.892113,1.353481,-1.506572,-0.067170,1.578803,-0.720089,-2.175227,0.899974,1.710468,0.903875
92378,2.149170,0.958080,1.822630,2.362643,-2.870665,-0.834484,-0.425472,2.726472,-0.294858,1.017502,...,2.031500,-1.959003,1.554847,-1.953300,2.565938,-0.250885,-0.816298,0.569747,-2.198135,0.988628
92379,1.570396,0.003341,2.096896,2.416096,-1.622049,1.666637,0.522704,-0.943092,-0.752680,-0.382468,...,0.612526,-2.405634,-1.543913,-1.089296,-0.936216,1.509843,0.312548,-2.355660,0.648719,0.726323


### 생성된 Doc2Vec 모델 테스트
- 임의의 뉴스 기사 혹은 단어를 통해서. 주어진 단어나 기사와 유사한 기사를 선별

In [381]:
# 모델 테스트를 위한 문장 또는 단어
text = """아파트"""

In [382]:
# 문장을 분석에 적합하게 전처리
text = get_only_korean_words(text)

In [383]:
# 문장을 토큰화 한 뒤 bigram 적용
tokens = noun_tokenizer(text)
bi_tokens = bigram[tokens]

# 학습된 모델을 바탕으로 token을 vector로 변환.
vectors = model.infer_vector(bi_tokens)

In [384]:
vectors

array([ 0.07285539, -0.01572077,  0.2354764 ,  0.26801646, -0.18391515,
       -0.07779261,  0.04362204,  0.1562555 , -0.01589913, -0.06036898,
       -0.01445667, -0.06893662, -0.05849586,  0.11434337,  0.07109629,
       -0.12782629, -0.05193966,  0.00632897,  0.07440759,  0.01445634,
        0.025059  , -0.10765764, -0.09207706,  0.04392311, -0.08492379,
        0.11964105, -0.08197746, -0.2556025 ,  0.14936033, -0.08145498,
        0.16005103, -0.0298384 , -0.06301613,  0.11664827,  0.17928931,
        0.04186594,  0.07121489,  0.13632405,  0.04359313, -0.16712056,
        0.05076169, -0.17605624, -0.01482332, -0.1454616 , -0.10805127,
       -0.01211539, -0.08111916,  0.12730676, -0.01441539, -0.25662723,
        0.06187486, -0.10099513, -0.11646703, -0.08318481,  0.09316004,
        0.04463357, -0.03534319,  0.16368137, -0.02689965, -0.08928417,
        0.031379  , -0.0039164 , -0.0417647 ,  0.05202786, -0.12459277,
        0.06188644, -0.07137702,  0.05568914, -0.2346649 , -0.04

In [385]:
# most_similar 메소드를 이용해서, 주어진 벡터와 유사한 기사들을 가져옴.
# topn=50을 통해 유사한 50개의 기사를 가져옴
model.dv.most_similar(vectors, topn=50)


# 각 값은 (기사의 인덱스, cosine_similarity값)

[(55560, 0.6638870239257812),
 (80223, 0.6071605682373047),
 (33641, 0.5942407250404358),
 (22816, 0.592218279838562),
 (316, 0.5910558104515076),
 (78180, 0.5893045663833618),
 (9077, 0.5874899625778198),
 (32053, 0.585460901260376),
 (33653, 0.582074761390686),
 (89773, 0.5743768811225891),
 (38145, 0.5725792646408081),
 (84992, 0.5713862776756287),
 (87212, 0.5696488618850708),
 (21121, 0.5687926411628723),
 (63378, 0.5686010718345642),
 (2470, 0.5668961405754089),
 (33114, 0.5661178231239319),
 (76299, 0.5645257830619812),
 (19615, 0.5644708275794983),
 (17626, 0.5639383792877197),
 (42907, 0.5610034465789795),
 (26589, 0.560885488986969),
 (75616, 0.559609591960907),
 (92356, 0.5580670237541199),
 (14335, 0.5556120872497559),
 (6210, 0.5547766089439392),
 (59648, 0.5544096827507019),
 (57136, 0.5525614619255066),
 (84948, 0.551805853843689),
 (21679, 0.5516817569732666),
 (35538, 0.5513553023338318),
 (71814, 0.5499094128608704),
 (4282, 0.5496848821640015),
 (29916, 0.54675853252

In [386]:
score = 0.5

# 유사한 기사 목록을 뽑아오기.
# 위의 out값에서 cosine_similarity 값이 score(여기서는 0.5) 보다 클 경우 index를 리스트에 담음.
sim_list = [i[0] for i in model.dv.most_similar(vectors, topn=50) if i[1] >= score]

In [387]:
# 유사 기사의 인덱스를 기사 DataFrame에 넣어서 기사 내용 가져오기.
sub_df = main_df.loc[sim_list]

In [388]:
# 논조 분류를 위해서 index를 리셋해줌.
sub_df = sub_df.reset_index(drop=True)

In [389]:
sub_df

,일자,제목,URL,본문,deleted,delete_photo,본문키워드
0,20211022,[사진] 수도권 아파트값 상승률 2주째 둔화,https://www.joongang.co.kr/article/25017068,수도권 아파트값 상승률 주째 둔화 수도권 아파트값 상승률이 주 ...,0.0,0.0,"[수도, 아파트, 상승, 둔화, 수도, 아파트, 상승, 연속, 둔화, 한국, 부동,..."
1,20210831,7월 주택 매매량 전년대비 37.1% 감소,http://biz.khan.co.kr/khan_art_view.html?artid...,서울 동작구 상공에서 바라본 도심 김창길 기자 ...,0.0,0.0,"[서울, 동작구, 상공, 도심, 김창길, 지난달, 주택, 매매, 국토, 교통부, 발..."
2,20210624,아파트서 60대 온몸 흉기 찔려 숨진 채 발견 남동생 긴급체포,http://www.seoul.co.kr/news/newsView.php?id=20...,서울의 한 아파트에서 대 노인이 머리 등 온몸을 흉기에 수차례 ...,0.0,0.0,"[서울, 아파트, 노인, 머리, 온몸, 흉기, 수차례, 사건, 발생, 가운데, 경찰..."
3,20210909,군포서 50대 의붓딸이 어머니 살해 후 투신해 숨져,http://www.seoul.co.kr/news/newsView.php?id=20...,경기 군포시 산본로 군포경찰서 전경 대 딸이 의붓어머니를 ...,0.0,0.0,"[경기, 군포시, 산본, 군포, 경찰서, 전경, 의붓어머니, 흉기, 살해, 투신, ..."
4,20211026,도심형 주거브랜드 '루미니' 안착,http://www.naeil.com/news_view/?id_art=402729,롯데건설이 시공한 오피스텔 고양 화정 루미니 조감도 가 완판됐다 롯데건설은 경...,0.0,0.0,"[롯데, 건설, 시공, 오피스텔, 고양, 화정, 미니, 조감도, 롯데, 건설, 경기..."
5,20210917,"""주식으로 돈 벌어 아파트 산 사람 1년 새 3배 증가""",http://biz.khan.co.kr/khan_art_view.html?artid...,서울 송파구 롯데타워 서울 스카이 전망대에서 내려다본 서울 시내에 시...,0.0,0.0,"[서울, 송파구, 롯데, 타워, 서울, 스카이, 망대, 서울, 시내, 시민, 주거,..."
6,20210507,세운 푸르지오 헤리시티 281가구 분양,http://www.naeil.com/news_view/?id_art=385421,대우건설은 서울 중구 인현동 가에 짓는 세운 푸르지오 헤리시티 주상복합아파트의 ...,0.0,0.0,"[대우건설, 서울, 중구, 인현동, 푸르지오, 헤리, 시티, 주상, 복합, 아파트,..."
7,20210706,서울 6억 이하 아파트 셋 중 하나 ‘실종’,http://www.seoul.co.kr/news/newsView.php?id=20...,시세 급등에 개월 새 만 가구 줄어중저가 주택 비중도 ...,0.0,0.0,"[시세, 급등, 개월, 어중, 주택, 비중, 상반기, 서울, 중저가, 주택, 기준,..."
8,20210624,서울 강서 아파트서 60대 온몸 흉기 찔려 숨진 채 발견,http://www.seoul.co.kr/news/newsView.php?id=20...,서울의 한 아파트에서 대 노인이 머리 등 온몸을 흉기에 수차례 ...,0.0,0.0,"[서울, 아파트, 노인, 머리, 온몸, 흉기, 수차례, 사망, 사건, 발생, 경찰,..."
9,20210531,수도권 아파트값 7억원 근접,http://news.khan.co.kr/kh_news/khan_art_view.h...,월 평균 매매가 억 만원 년 월 집계 이후 최고가 서울 전...,0.0,0.0,"[평균, 매매, 집계, 이후, 최고, 서울, 전세, 수도, 아파트, 평균, 매매, ..."


## 3. 유사 기사들을 바탕으로 각 기사별 논조(차이점) 분류
- 한 토픽의 기사도 작성자에 따라 다른 관점을 취할 수 있음.
- 이러한 차이점이 존재하는지 TfidfVectorizer를 이용하여 확인.

### TfidfVectorizer를 이용해 각 뉴스기사에 동일하게 존재하는 토큰 제거
- 이미 Doc2Vec을 통해 유사한 토픽의 뉴스 기사를 가져왔기 때문에 차이점을 분류하는데 불필요한 토큰들 제거
- 즉, 뉴스 기사들 사이의 교집합을 제거

In [390]:
# TfidfVectorizer를 이용해 각 뉴스기사에 존재하는 토큰 갯수 확인.
# min_df를 전체 뉴스기사 수의 절반을 주어, 많은 기사들에 존재하는 토큰들만 확인.
tfidf_vect = TfidfVectorizer(tokenizer=noun_tokenizer, min_df = round(len(sub_df)/2))
matrix = tfidf_vect.fit_transform(sub_df['본문']).toarray()

In [391]:
matrix_df = pd.DataFrame(matrix, columns=tfidf_vect.get_feature_names_out())

In [392]:
matrix_df

,서울,아파트,주택
0,0.505967,0.862553,0.000000
1,0.101507,0.259568,0.960375
2,0.616071,0.787690,0.000000
3,0.000000,1.000000,0.000000
4,0.000000,0.000000,1.000000
5,0.408363,0.278464,0.869308
6,0.334494,0.855350,0.395589
7,0.382242,0.895992,0.226029
8,0.616071,0.787690,0.000000
9,0.585007,0.623311,0.518893


In [393]:
# 0.0이 아닌 값들(해당 뉴스기사에 존재하는 토큰)의 숫자를 세고, 전체 뉴스기사 수 80% 이상에서 관찰된 토큰은 제거.
# 50개의 뉴스기사가 있다고 가정하면, 40개 이상의 문서에서 발견되는 토큰 제거.
(matrix_df != 0.0).sum()

서울     37
아파트    45
주택     29
dtype: int64

In [394]:
# 전체의 80% 이상에서 발견되는 토큰들을 deprecated_words 리스트에 담아줌
deprecated_words = (matrix_df != 0.0).sum()[((matrix_df != 0.0).sum() >= round(len(sub_df) * 0.8))].index

In [395]:
# 위 기사들에서는 아파트 외에 자주 나오는 토큰이 발견되지 않음.
deprecated_words

Index(['아파트'], dtype='object')

In [396]:
# deprecated_words를 적용하기 위한 함수
def make_small_token(temp_list) :
    return [token for token in temp_list if token not in deprecated_words]

# apply메소드를 이용해, 각 뉴스기사의 토큰들에서 제거.
sub_df['small_token'] = sub_df['본문키워드'].apply(make_small_token)

In [397]:
# 본문키워드와 small_token을 비교했을 때 아파트 토큰만이 제거됨.
sub_df

,일자,제목,URL,본문,deleted,delete_photo,본문키워드,small_token
0,20211022,[사진] 수도권 아파트값 상승률 2주째 둔화,https://www.joongang.co.kr/article/25017068,수도권 아파트값 상승률 주째 둔화 수도권 아파트값 상승률이 주 ...,0.0,0.0,"[수도, 아파트, 상승, 둔화, 수도, 아파트, 상승, 연속, 둔화, 한국, 부동,...","[수도, 상승, 둔화, 수도, 상승, 연속, 둔화, 한국, 부동, 산원, 서울, 경..."
1,20210831,7월 주택 매매량 전년대비 37.1% 감소,http://biz.khan.co.kr/khan_art_view.html?artid...,서울 동작구 상공에서 바라본 도심 김창길 기자 ...,0.0,0.0,"[서울, 동작구, 상공, 도심, 김창길, 지난달, 주택, 매매, 국토, 교통부, 발...","[서울, 동작구, 상공, 도심, 김창길, 지난달, 주택, 매매, 국토, 교통부, 발..."
2,20210624,아파트서 60대 온몸 흉기 찔려 숨진 채 발견 남동생 긴급체포,http://www.seoul.co.kr/news/newsView.php?id=20...,서울의 한 아파트에서 대 노인이 머리 등 온몸을 흉기에 수차례 ...,0.0,0.0,"[서울, 아파트, 노인, 머리, 온몸, 흉기, 수차례, 사건, 발생, 가운데, 경찰...","[서울, 노인, 머리, 온몸, 흉기, 수차례, 사건, 발생, 가운데, 경찰, 사건,..."
3,20210909,군포서 50대 의붓딸이 어머니 살해 후 투신해 숨져,http://www.seoul.co.kr/news/newsView.php?id=20...,경기 군포시 산본로 군포경찰서 전경 대 딸이 의붓어머니를 ...,0.0,0.0,"[경기, 군포시, 산본, 군포, 경찰서, 전경, 의붓어머니, 흉기, 살해, 투신, ...","[경기, 군포시, 산본, 군포, 경찰서, 전경, 의붓어머니, 흉기, 살해, 투신, ..."
4,20211026,도심형 주거브랜드 '루미니' 안착,http://www.naeil.com/news_view/?id_art=402729,롯데건설이 시공한 오피스텔 고양 화정 루미니 조감도 가 완판됐다 롯데건설은 경...,0.0,0.0,"[롯데, 건설, 시공, 오피스텔, 고양, 화정, 미니, 조감도, 롯데, 건설, 경기...","[롯데, 건설, 시공, 오피스텔, 고양, 화정, 미니, 조감도, 롯데, 건설, 경기..."
5,20210917,"""주식으로 돈 벌어 아파트 산 사람 1년 새 3배 증가""",http://biz.khan.co.kr/khan_art_view.html?artid...,서울 송파구 롯데타워 서울 스카이 전망대에서 내려다본 서울 시내에 시...,0.0,0.0,"[서울, 송파구, 롯데, 타워, 서울, 스카이, 망대, 서울, 시내, 시민, 주거,...","[서울, 송파구, 롯데, 타워, 서울, 스카이, 망대, 서울, 시내, 시민, 주거,..."
6,20210507,세운 푸르지오 헤리시티 281가구 분양,http://www.naeil.com/news_view/?id_art=385421,대우건설은 서울 중구 인현동 가에 짓는 세운 푸르지오 헤리시티 주상복합아파트의 ...,0.0,0.0,"[대우건설, 서울, 중구, 인현동, 푸르지오, 헤리, 시티, 주상, 복합, 아파트,...","[대우건설, 서울, 중구, 인현동, 푸르지오, 헤리, 시티, 주상, 복합, 사이버,..."
7,20210706,서울 6억 이하 아파트 셋 중 하나 ‘실종’,http://www.seoul.co.kr/news/newsView.php?id=20...,시세 급등에 개월 새 만 가구 줄어중저가 주택 비중도 ...,0.0,0.0,"[시세, 급등, 개월, 어중, 주택, 비중, 상반기, 서울, 중저가, 주택, 기준,...","[시세, 급등, 개월, 어중, 주택, 비중, 상반기, 서울, 중저가, 주택, 기준,..."
8,20210624,서울 강서 아파트서 60대 온몸 흉기 찔려 숨진 채 발견,http://www.seoul.co.kr/news/newsView.php?id=20...,서울의 한 아파트에서 대 노인이 머리 등 온몸을 흉기에 수차례 ...,0.0,0.0,"[서울, 아파트, 노인, 머리, 온몸, 흉기, 수차례, 사망, 사건, 발생, 경찰,...","[서울, 노인, 머리, 온몸, 흉기, 수차례, 사망, 사건, 발생, 경찰, 수사, ..."
9,20210531,수도권 아파트값 7억원 근접,http://news.khan.co.kr/kh_news/khan_art_view.h...,월 평균 매매가 억 만원 년 월 집계 이후 최고가 서울 전...,0.0,0.0,"[평균, 매매, 집계, 이후, 최고, 서울, 전세, 수도, 아파트, 평균, 매매, ...","[평균, 매매, 집계, 이후, 최고, 서울, 전세, 수도, 평균, 매매, 가격, 돌..."


### small_token(너무 많이 존재하는 토큰이 삭제된 토큰목록)과 KMeans을 이용해서 Clustering

In [398]:
# 각 기사를 벡터화 하기 위해서 TfidfVectorizer 이용
# 문자열이 아닌 토큰으로 학습하기 위해서 tokenizer를 지정하지 않음(띄어쓰기를 바탕으로 토큰을 자동 구분)
split_tfidf = TfidfVectorizer()

# TfidfVectorizer의 경우 문장(String 타입의 텍스트)이 들어가야하나,
# small_token(토큰이 들어가있는 리스트)를 통해 분석하기 위해서, ' '.join(list) 메소드를 이용해서 리스트를 문자열로 바꿔줌
split_matrix = split_tfidf.fit_transform(sub_df['small_token'].apply(lambda x : ' '.join(x))).toarray()

In [399]:
split_matrix_df = pd.DataFrame(split_matrix, columns=split_tfidf.get_feature_names_out())

In [400]:
split_matrix_df

,가격,가격대,가결,가계,가구,가능,가량,가상,가스총,가운데,...,회견,회로,회천,횡보,효과,후원,휴가철,휴일,흉기,흔적
0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.100379,...,0.000000,0.116919,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.116919,0.000000
3,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.115378,0.000000
4,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.00000,0.000000,0.197685,0.000000,0.000000,0.17122,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.073731,0.00000,0.000000,0.297947,0.000000,0.051940,0.00000,0.000000,0.099234,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.144979,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.144979,0.000000
9,0.300649,0.000000,0.00000,0.074406,0.047497,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### 감성분석을 위한 함수

In [401]:
def senti_dict(wordname):
    with open('SentiWord_info.json', encoding='utf-8-sig', mode='r') as f:
        data = json.load(f)
    result = ['None','None']
    for i in range(0, len(data)):
        if data[i]['word'] == wordname:
            result.pop()
            result.pop()
            result.append(data[i]['word_root'])
            result.append(data[i]['polarity'])

    r_word = result[0]
    s_word = result[1]

    return r_word, s_word

### 각 뉴사기사의 토큰들을 감성분석기에 넣어서 점수화.
- -3점 이하는 부정
- -2 ~ 2점은 중립
- 3점 이상은 긍정으로 분류

In [402]:
%%time
senti_points = []
for tokens in sub_df['본문키워드'] :
    point = 0
    for token in tokens :
        res = senti_dict(token)[1]
        if res == 'None' :
            continue
        point = point + int(res)
    if point < -2 :
        senti_points.append(-1)
    elif point > 2 :
        senti_points.append(1)
    else :
        senti_points.append(0)

CPU times: user 37.4 s, sys: 2.04 s, total: 39.4 s
Wall time: 39.4 s


In [403]:
# 각 기사별 감성분석 점수를 뉴스기사가 벡터화된 matrix에 추가.
split_matrix_df['점수'] = senti_points

In [404]:
# 부정, 중립, 긍정 기사를 구분하기 위해서 one-hot 처리
split_matrix_df = pd.get_dummies(split_matrix_df, columns=['점수'])

In [405]:
split_matrix_df

,가격,가격대,가결,가계,가구,가능,가량,가상,가스총,가운데,...,횡보,효과,후원,휴가철,휴일,흉기,흔적,점수_-1,점수_0,점수_1
0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0
1,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0
2,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.100379,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.116919,0.000000,0,1,0
3,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.115378,0.000000,0,1,0
4,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0
5,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0
6,0.000000,0.000000,0.00000,0.000000,0.197685,0.000000,0.000000,0.17122,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0
7,0.000000,0.073731,0.00000,0.000000,0.297947,0.000000,0.051940,0.00000,0.000000,0.099234,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0
8,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.144979,0.000000,0,1,0
9,0.300649,0.000000,0.00000,0.074406,0.047497,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,1


### KMeans를 이용해 군집화

In [406]:
# 기사를 분류하기 위해 n_clusters는 3으로 지정
kmm = KMeans(n_clusters = 3)
kmm_preds = kmm.fit_predict(split_matrix)

In [407]:
pd.Series(kmm_preds).value_counts()

1    19
0    16
2    15
dtype: int64

In [408]:
final_df = sub_df[['제목', '본문']]
final_df['kmm_cl'] = kmm_preds

/var/folders/fy/f7hm_xxs1bb1k3q0vxk7zc_w0000gn/T/ipykernel_2140/667989990.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['kmm_cl'] = kmm_preds


In [409]:
# 대체적으로 아파트에서 발생한 사고에 대해 분류
final_df[final_df.kmm_cl == 0]

,제목,본문,kmm_cl
2,아파트서 60대 온몸 흉기 찔려 숨진 채 발견 남동생 긴급체포,서울의 한 아파트에서 대 노인이 머리 등 온몸을 흉기에 수차례 ...,0
3,군포서 50대 의붓딸이 어머니 살해 후 투신해 숨져,경기 군포시 산본로 군포경찰서 전경 대 딸이 의붓어머니를 ...,0
8,서울 강서 아파트서 60대 온몸 흉기 찔려 숨진 채 발견,서울의 한 아파트에서 대 노인이 머리 등 온몸을 흉기에 수차례 ...,0
10,충남 공주서 아내 살해한 50대 검거,한겨레 자료사진 충남 공주경찰서는 일 아내를 숨지게 한 혐의 살...,0
13,“왜 안 만나줘” 전처 찌르고 도주한 70대 경찰 추적 중,만나자는 요구 계속 거절하자 집으로 찾아가 경남 사천에서 이혼한 ...,0
19,[속보] ‘왕릉 아파트 승인’ 인천 서구청 등 압수수색,...,0
20,“어머니 형 연락 안 된다”신고에 강서구 다세대주택서 3명 주검 발견,한겨레 자료사진 서울 강서구 한 다세대주택에서 일 어머니와 아...,0
21,고양 아파트 12층서 9세 초등생 떨어져 숨져,경찰 학대 학교 폭력 여부 등 조사 중 경기 고양시의 한 아파트에...,0
27,60대 할아버지 손자 2명과 아파트 옥상서 추락 극단적 선택 추정,중앙포토 지난 일 오후 시쯤 부산의 한 아파트 화단에 대 ...,0
29,광주 아파트서 묶여 있는 60대 여성 시신 발견 타살 정황,사망 사건 자료사진 픽사베이 광주의 한 아파트에서 타살로 의심...,0


In [410]:
# 대체적으로 아파트 가격과 관련된 기사들을 분류
final_df[final_df.kmm_cl == 1]

,제목,본문,kmm_cl
0,[사진] 수도권 아파트값 상승률 2주째 둔화,수도권 아파트값 상승률 주째 둔화 수도권 아파트값 상승률이 주 ...,1
7,서울 6억 이하 아파트 셋 중 하나 ‘실종’,시세 급등에 개월 새 만 가구 줄어중저가 주택 비중도 ...,1
9,수도권 아파트값 7억원 근접,월 평균 매매가 억 만원 년 월 집계 이후 최고가 서울 전...,1
11,"서울 아파트값, 올 상반기에만 지난해 1년치 넘게 올라",서울 송파구 롯데타워 서울 스카이 전망대에서 내려다본 서울 시내에 시...,1
12,"수도권 아파트 7억 돌파, 8개월만에 1억 뛰어",수도권 아파트 평균 매매가가 개월만에 억원이 오르며 억원을 넘어섰다 ...,1
14,"서울 상위 20% 집값, 文 이전 12억원 → 文 이후 22억원",서울 강남구 아파트 단지의 모습 뉴스 수도권에서 집값 상위 ...,1
15,"외지인 아파트 매입 28.1%, 역대최고",올해 월 해당지역에 거주하지 않는 외지인의 아파트 매입 비중이 역대 최고치를 ...,1
16,수도권 아파트값 7억 돌파 8개월 만에 1억↑,이달 서울 억 강북지역 억대전국 평균 매매가격은 억원 넘어 ...,1
18,1억5천만원 넘게 오른 서울 평균 아파트값...12억원 육박,...,1
26,"아파트 266채, 한사람이 쓸었다 3년간 10채↑ 구매자 965명",서울 송파구 주공 단지를 비롯한 서울 시내 아파트단지 모습 연합뉴스 ...,1


In [411]:
# 아파트 거래량과 관련된 기사를 분류.
final_df[final_df.kmm_cl == 2]

,제목,본문,kmm_cl
1,7월 주택 매매량 전년대비 37.1% 감소,서울 동작구 상공에서 바라본 도심 김창길 기자 ...,2
4,도심형 주거브랜드 '루미니' 안착,롯데건설이 시공한 오피스텔 고양 화정 루미니 조감도 가 완판됐다 롯데건설은 경...,2
5,"""주식으로 돈 벌어 아파트 산 사람 1년 새 3배 증가""",서울 송파구 롯데타워 서울 스카이 전망대에서 내려다본 서울 시내에 시...,2
6,세운 푸르지오 헤리시티 281가구 분양,대우건설은 서울 중구 인현동 가에 짓는 세운 푸르지오 헤리시티 주상복합아파트의 ...,2
17,[단독]서울 5억 미만 아파트 거래 5년 새 3분의1 수준으로,서울에서 실거래가가 억원 미만 매매거래 비중이 최근 년여 새 분의 수준으...,2
22,서초구 '반포센트럴자이' 4년 만에 19억→45억으로,약 년전 분양된 서울 아파트 개 단지의 실거래가격이 분양 당시 대비 평균 ...,2
23,"택배노조, 6일 총파업 투표...가결시 11일 총파업",지난 일 서울 강서구에서 기자회견 중인 택배노조 연합뉴스 ...,2
24,"지난해 주택 분양 늘고, 준공 줄어",지난해 주택 착공과 분양은 년보다 늘어난 반면 인허가와 준공은 줄어든 것으...,2
25,"5월 누계 인허가.착공.분양 늘고, 준공 줄어",올들어 월까지 전국 주택 인허가 착공 분양 물량은 지난해보다 늘어난 반면 준...,2
30,"2017년 분양된 서울 아파트, 4년 만에 평균 10억원 ‘껑충’",년 분양된 서울 잠원동 반포센트럴자이 본보기집 한겨레 자료...,2
